In [ ]:
import numpy as np
import pandas as pd

import torch.nn as nn                 # the torch module to implement the Neural Networks
import torch.nn.parallel              # for parallel computations
import torch.optim as optim           # for optimizers
import torch.utils.data               # tools
from torch.autograd import Variable   # for Stochastic Gradient Descent

In [ ]:
reviews = pd.read_csv('data/attractions/reviews/attractions_reviews_clean.csv')
reviews.head()

In [ ]:
details = pd.read_csv('data/attractions/details/attractions_details_clean.csv')
details.head()

In [ ]:
reviews.drop(['review', 'review_date', 'user'], axis=1, inplace=True)
reviews.head()


In [ ]:
# Pivot the ratings data to get a user-movie rating matrix
reviews_matrix = reviews.pivot_table(index='user_id', columns='attraction_id', values='rating', aggfunc='mean')
reviews_matrix.fillna(0, inplace=True)

In [ ]:
reviews_matrix

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test = train_test_split(reviews_matrix.values, test_size=0.2, random_state=42)

In [ ]:
# Normalize the training data to have zero mean and unit variance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_train_norm

# Convert pivot table DataFrame to binary matrix format using threshold
# binary_matrix = (pivot_table.fillna(0) > 3).astype(np.int32).values

In [ ]:
# Define the number of visible and hidden units for the RBM model
visible_units = X_train_norm.shape[1]
hidden_units = 100
learning_rate = 0.01
batch_size = 100
epochs = 10

print(visible_units, hidden_units)

In [ ]:
train_set = torch.FloatTensor(X_train_norm)
test_set = torch.FloatTensor(X_test)

In [ ]:
import tensorflow as tf
from rbm_model import RBM

# Train RBM model
rbm_model = RBM(visible_units, hidden_units)
# optimizer = tf.keras.optimizers.Adam(learning_rate)

# for epoch in range(epochs):
#     loss_update = 0
#     for i in range(0, X_train_norm.shape[0], batch_size):
#         batch = X_train_norm[i:i+batch_size]
#         with tf.GradientTape() as tape:
#             reconstructed_batch = rbm_model(batch)
#             loss = tf.reduce_mean(tf.square(batch - reconstructed_batch))
#             loss_update = loss 
#         gradients = tape.gradient(loss, rbm_model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients, rbm_model.trainable_variables))
#     print("Epoch: ", epoch, "Loss: ", loss_update.numpy())


In [ ]:
# Save the trained model for future use
#rbm_model.save('weight/rbm_weight_model.h5')

# Example usage
# Assume you have trained an RBM model named "model"
# and you have a test dataset "test_data"
# You can save the model using the following code:
checkpoint_path = "weight/rbm_weight_model"
checkpoint = tf.train.Checkpoint(model=rbm_model)
checkpoint.write(checkpoint_path)

# You can then load the model using the following code:
loaded_checkpoint = tf.train.Checkpoint(model=RBM(visible_units, hidden_units))
loaded_checkpoint.restore(checkpoint_path)
new_rbm_model = loaded_checkpoint.model


In [ ]:
def evaluate(model, test_matrix):
    aps = []
    for user_id in range(test_matrix.shape[0]):
        user_ratings = test_matrix[user_id]
        rated_items = np.where(user_ratings > 0)[0]
        if len(rated_items) > 0:
            hidden_representation = model.sample_hidden(tf.constant([user_ratings], dtype=tf.float32))
            predicted_ratings = model.sample_visible(hidden_representation)
            recommended_items = (-predicted_ratings.numpy()).argsort()[0]
            ap = average_precision(recommended_items, rated_items)
            aps.append(ap)
    map_score = np.mean(aps)
    return map_score

def average_precision(recommended_items, relevant_items):
    ap = 0
    hits = 0
    for i, item in enumerate(recommended_items):
        if item in relevant_items:
            hits += 1
            ap += hits / (i + 1)
    if hits > 0:
        ap /= hits
    return ap

# Use the trained RBM model to predict the ratings for the test set
X_test_norm = scaler.transform(X_test)

print(evaluate(new_rbm_model, X_test_norm))

In [ ]:
# Make hotel recommendations for a given user
user_id = 0
user_ratings = X_train_norm[user_id]
hidden_representation = new_rbm_model.sample_hidden(tf.constant([user_ratings], dtype=tf.float32))
predicted_ratings = new_rbm_model.sample_visible(hidden_representation)
print((-predicted_ratings.numpy()).argsort()[0])
recommendations = (-predicted_ratings.numpy()).argsort()[0][:100]

print(recommendations)

In [ ]:
top_n = 10
scores = new_rbm_model.predict(tf.constant([user_ratings], dtype=tf.float32))
scores_final = scores.tolist()[0]
print(scores_final)